In [58]:
from __future__ import absolute_import, division, print_function

from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import os

from tensorflow.keras import layers
from tensorflow.keras import optimizers 

import pandas as pd
import numpy as np

tf.VERSION

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_data = image_generator.flow_from_directory("flower_photos",target_size=(32,32))
validation_data = image_generator.flow_from_directory("flower_photos_test",target_size=(32,32))

for image_batch,label_batch in train_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []
    self.epoch_val_loss = []
    self.epoch_val_acc = []
    
  def on_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
  def on_epoch_end(self, epoch, logs=None):
    self.epoch_val_loss.append(logs['val_loss'])
    self.epoch_val_acc.append(logs['val_acc'])

runs = [2, 3]
activation = [0, 0, 1, 1, 2, 2]
lossF = ['categorical_crossentropy', 'mean_squared_error', 'categorical_crossentropy', 'mean_squared_error', 'categorical_crossentropy', 'mean_squared_error']
saveName = ['Tanh', 'MSETanh', 'RELU', 'MSERELU', 'LeakyRELU', 'MSELeakyRELU']

Found 3303 images belonging to 5 classes.
Found 367 images belonging to 5 classes.
Image batch shape:  (32, 32, 32, 3)
Label batch shape:  (32, 5)


In [ ]:
for id in runs:
    model = None
    if (activation[id] == 0):
        # CNN 2
        model = tf.keras.Sequential([
          layers.Conv2D(32, [3, 3], (1, 1), 'valid', input_shape=(image_batch.shape[1], image_batch.shape[2], image_batch.shape[3])),
          layers.Activation('tanh'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(64, [3, 3], (1, 1), 'valid'),
          layers.Activation('tanh'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(128, [3, 3], (1, 1), 'valid'),
          layers.Activation('tanh'),
          # layers.LeakyReLU(alpha=0.3),
          layers.Flatten(),

          layers.Dense(train_data.num_classes, activation='softmax')
        ])
    if (activation[id] == 1):
        # CNN 2
        model = tf.keras.Sequential([
          layers.Conv2D(32, [3, 3], (1, 1), 'valid', input_shape=(image_batch.shape[1], image_batch.shape[2], image_batch.shape[3])),
          layers.Activation('relu'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(64, [3, 3], (1, 1), 'valid'),
          layers.Activation('relu'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(128, [3, 3], (1, 1), 'valid'),
          layers.Activation('relu'),
          # layers.LeakyReLU(alpha=0.3),
          layers.Flatten(),

          layers.Dense(train_data.num_classes, activation='softmax')
        ])
    if (activation[id] == 2):
        # CNN 2
        model = tf.keras.Sequential([
          layers.Conv2D(32, [3, 3], (1, 1), 'valid', input_shape=(image_batch.shape[1], image_batch.shape[2], image_batch.shape[3])),
          # layers.Activation('relu'),
          layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(64, [3, 3], (1, 1), 'valid'),
          # layers.Activation('relu'),
          layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(128, [3, 3], (1, 1), 'valid'),
          # layers.Activation('relu'),
          layers.LeakyReLU(alpha=0.3),
          layers.Flatten(),

          layers.Dense(train_data.num_classes, activation='softmax')
        ])
    model.compile(
        optimizer=tf.train.AdamOptimizer(), 
        loss=lossF[id],
        metrics=['accuracy'])
    
    batch_stats = CollectBatchStats()
    model.fit_generator(train_data, epochs=4, 
        callbacks = [batch_stats],
        validation_data = validation_data)
    
    results = pd.DataFrame({"losses":batch_stats.batch_losses, "accuracy":batch_stats.batch_acc})
    results.to_csv("saved_results/cnn"+saveName[id]+"Data.csv", ',')
    results = pd.DataFrame({"losses":batch_stats.epoch_val_loss, "accuracy":batch_stats.epoch_val_acc})
    results.to_csv("saved_results/cnn"+saveName[id]+"ValData.csv", ',')

In [59]:
categories = len(np.unique(data.y_train))
processed_data = preprocess(data, categories)

NameError: name 'data' is not defined

In [72]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
actFunc = ['softmax','relu','leaky','tanh']
lossFunc = ['mean_squared_error','categorical_crossentropy']
imSizes = [32,64]

def getData(size):
    t_data = image_generator.flow_from_directory("flower_photos",color_mode="grayscale",target_size=(size,size))
    v_data = image_generator.flow_from_directory("flower_photos_test",color_mode="grayscale",target_size=(size,size))
    return t_data, v_data

def plainVanilla(images, labels, activ, los):
    #print (images.shape[1], images.shape[2], images.shape[3])
    if actFunc[k] == 'leaky':
        model = tf.keras.Sequential([
            layers.Flatten(input_shape=(images.shape[1], images.shape[2], images.shape[3])),
            layers.Dense(1024),
            layers.LeakyReLU(alpha=0.3),
            layers.Dense(512),
            layers.LeakyReLU(alpha=0.3),
            layers.Dense(labels, activation='softmax')
        ])
    else:
        model = tf.keras.Sequential([
            layers.Flatten(input_shape=(images.shape[1], images.shape[2], images.shape[3])),
            layers.Dense(1024, activation=activ),
            #layers.Dropout(0.2),
            layers.Dense(512, activation=activ),
            #layers.Dropout(0.2),
            layers.Dense(labels, activation='softmax')
        ])
    model.compile(
        optimizer=tf.train.AdamOptimizer(), 
        loss=los,
        metrics=['accuracy'])
    return model

def trainModel(model, tr_dat, val_dat):
    batch_stats = CollectBatchStats()
    model.fit_generator(tr_dat, epochs=epoch, 
        callbacks = [batch_stats],         
        validation_data = val_dat)
    return callbacks

    

#initialisation
epoch = 15

savedMods = []
runs = 1
for i in range((len(imSizes)-1)):
    train_dat, validation_dat = getData(imSizes[i])
    
    for image_batch,label_batch in train_dat:
        print("Image batch shape: ", image_batch.shape)
        print("Label batch shape: ", label_batch.shape)
        break
    
    for j in range(len(lossFunc)-1):
        for k in range(len(actFunc)-1):
            print ('This is run: '+ str(runs))
            print ('\n \n')
            print ('Image size: '+ str(imSizes[i]) + ' * ' + str(imSizes[i]))
            print ('Loss function: '+lossFunc[j])
            print ('Activation function: '+actFunc[k])
            print ( '\n')
            model = plainVanilla(image_batch, train_dat.num_classes, actFunc[k], lossFunc[j])
            batch_stats = CollectBatchStats()
            mod = model.fit(train_dat,
                            epochs=epoch, 
                            validation_data=validation_dat,
                            callbacks=[batch_stats]
                           )
            savedMods.append([imSizes[i], actFunc[k], lossFunc[j], mod])
            runs += 1
            
    

Found 3303 images belonging to 5 classes.
Found 367 images belonging to 5 classes.
Image batch shape:  (32, 32, 32, 1)
Label batch shape:  (32, 5)
This is run: 1

 

Image size: 32 * 32
Loss function: mean_squared_error
Activation function: softmax
Epoch 1/15
 94/104 [==========================>...] - ETA: 1s - loss: 0.1597 - acc: 0.2400

KeyboardInterrupt: 

# extra:

In [51]:
# pak van elke soort een plaatje
soorten = os.listdir('flower_photos')[0:5]
simg = []
for n in range(5):
    simg.append(Image('flower_photos/'+soorten[n]+'/'+os.listdir('flower_photos/'+soorten[n])[5]))
print(soorten)

['roses', 'sunflowers', 'daisy', 'dandelion', 'tulips']


In [56]:
len(train_data)
len(validation_data)

12

In [57]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_data = image_generator.flow_from_directory("flower_photos",target_size=(32,32))
validation_data = image_generator.flow_from_directory("flower_photos_test",target_size=(32,32))

Found 3303 images belonging to 5 classes.
Found 367 images belonging to 5 classes.


In [68]:
range(len(imSizes))

range(0, 2)